In [1]:
import pandas as pd
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
#from scripts.preprocess_cpv import process_cpv_data
from scripts.preprocess_categorielle import preprocess_categorielle, encoding_idAccordCadre
from scripts.preprocess_missing_values import clean_missing_values
from scripts.preprocess_cpv import add_cpv_hierarchy_column
from scripts.preprocess_numerique import log_transformation, scale_numerical_features

In [2]:
import pandas as pd
import numpy as np
import os
import sqlite3
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [3]:
db_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'datalab.sqlite')
db_path

'/Users/loicdogon/code/RonanB400/decp_ml/data/datalab.sqlite'

In [4]:
conn = sqlite3.connect(db_path)
db = conn.cursor()
query = f"""
SELECT *
FROM "data.gouv.fr.2022.clean"
"""

data = pd.read_sql_query(query, conn)

In [5]:
X_train, X_test = train_test_split(data, test_size=0.2, random_state=0)

In [6]:
X_train.to_csv(os.path.join(os.path.dirname(os.getcwd()), 'data', 'train.csv'), index=False)
X_test.to_csv(os.path.join(os.path.dirname(os.getcwd()), 'data', 'test.csv'), index=False)

In [7]:
df = pd.read_csv('../data/train.csv', encoding='utf-8')

In [10]:
df

,uid,id,nature,acheteur_id,acheteur_nom,acheteur_siren,titulaire_id,titulaire_typeIdentifiant,titulaire_nom,titulaire_siren,...,ccag,sousTraitanceDeclaree,typeGroupementOperateurs,tauxAvance,origineUE,origineFrance,lieuExecution_code,lieuExecution_typeCode,idAccordCadre,source_open_data
0,215706722000182024TH240,2024TH240,Marché,21570672200018,COMMUNE DE THIONVILLE,215706722.0,42153568300013,SIRET,TRAVAUX SOUTERRAINS ELECTRIQUES ET GAZIERS,421535683.0,...,Travaux,0.0,Pas de groupement,0.05,0.0,0.0,57100,Code postal,NaN,data.gouv.fr decp-2025-04.json
1,200052090000122023ELECF8,2023ELECF8,Marché,20005209000012,SYNDICAT INTERCOMMUNALD'ENERGIES DU DEPARTEMEN...,200052090.0,53791596900134,SIRET,CEGELEC RODEZ,537915969.0,...,Pas de CCAG,NaN,Pas de groupement,0.00,0.0,0.0,12032,Code postal,NaN,data.gouv.fr decp-2025-04.json
2,200058519001132023-0202,2023-0202,Marché,20005851900113,CA SAINT GERMAIN BOUCLES DE SEINE,200058519.0,42094822600048,SIRET,EUROVIA ILE DE FRANCE,420948226.0,...,Travaux,0.0,Conjoint,NaN,NaN,NaN,78230,Code postal,2023-0202,data.gouv.fr decp-2024.json
3,239740012000122023602099,2023602099,Marché,23974001200012,REGION REUNION,239740012.0,50067844600047,SIRET,TOPO SERVICES,500678446.0,...,Pas de CCAG,NaN,Pas de groupement,NaN,NaN,NaN,04,Code région,2023602099,data.gouv.fr decp-2024.json
4,2631001250001620241941,20241941,Marché,26310012500016,CENTRE HOSPITALIER UNIVERSITAIRE DE TOULOUSE,263100125.0,77566225700150,SIRET,SANOFI WINTHROP INDUSTRIE,775662257.0,...,Fournitures courantes et services,NaN,Pas de groupement,0.00,0.0,0.0,76,Code région,NaN,data.gouv.fr decp-2025-03.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239731,256701152000252024S0055MPAOC,2024S0055MPAOC,Marché,25670115200025,SYNDICAT MIXTE DES EAUX ET DE L'ASSAINISSEMENT...,256701152.0,50217952600038,SIRET,T&S TECHNOLOGY &STRATEGY I.T.,502179526.0,...,Pas de CCAG,0.0,Pas de groupement,NaN,NaN,NaN,44,Code région,2023S0156MAOOC,data.gouv.fr decp-2024.json
239732,216901231000112023O4484,2023O4484,Marché,21690123100011,COMMUNE DE LYON,216901231.0,30389008100126,SIRET,CARDEM,303890081.0,...,Pas de CCAG,1.0,Pas de groupement,NaN,NaN,NaN,69123,Code commune,NaN,data.gouv.fr decp-2024.json
239733,211202965000112024001L06,2024001L06,Marché,21120296500011,COMMUNE DE VIALA DU TARN,211202965.0,53920081600012,SIRET,MENUISERIE DURAND,539200816.0,...,Pas de CCAG,NaN,Pas de groupement,NaN,NaN,NaN,12490,Code postal,NaN,data.gouv.fr decp-2024.json
239734,2113010070001220240701,20240701,Marché,21130100700012,COMMUNE DE SAINT REMY DE PROVENCE,211301007.0,32344748200015,SIRET,SELE,323447482.0,...,Travaux,0.0,Conjoint,0.05,0.0,0.0,13210,Code postal,NaN,data.gouv.fr decp-2025-04.json


In [11]:
df.isnull().sum()

uid                               0
id                                0
nature                            0
acheteur_id                       0
acheteur_nom                    154
acheteur_siren                  137
titulaire_id                      0
titulaire_typeIdentifiant         0
titulaire_nom                  6037
titulaire_siren                 912
objet                             0
montant                           0
codeCPV                           0
procedure                         0
dureeMois                         6
dateNotification                  0
datePublicationDonnees            2
formePrix                         0
attributionAvance            139179
offresRecues                  81034
marcheInnovant               141146
ccag                              0
sousTraitanceDeclaree        115718
typeGroupementOperateurs          0
tauxAvance                   103643
origineUE                    104388
origineFrance                104388
lieuExecution_code          

In [9]:
from scripts.preprocess_final import create_preprocessing_pipeline_init

In [12]:
pipeline = create_preprocessing_pipeline_init(cat='marche_sim')


X_transformed = pipeline.fit_transform(df)

ModuleNotFoundError: No module named 'preprocess_cpv'